In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
pd.set_option('display.max_column',100)
pa = 'https://raw.githubusercontent.com/kshedden/statswpy-nhanes/master/merged/nhanes_2015_2016.csv'
df = pd.read_csv(pa)
df.head()

In [ ]:
sns.regplot(x='BMXLEG', y='BMXARML', data=df)

In [ ]:
sns.regplot(x='BMXLEG', y='BMXARML', data=df, fit_reg=False, scatter_kws={'alpha':0.3})

In [ ]:
sns.jointplot(x='BMXLEG',y='BMXARML', data=df)

In [ ]:
sns.jointplot(x='BMXLEG',y='BMXARML', data=df, kind='reg')

In [ ]:
sns.jointplot(x='BMXLEG',y='BMXARML', data=df, kind='scatter')

In [ ]:
sns.jointplot(x='BMXLEG',y='BMXARML', data=df, kind='resid')

In [ ]:
sns.jointplot(x='BMXLEG',y='BMXARML', data=df, kind='kde')

In [ ]:
sns.jointplot(x='BMXLEG',y='BMXARML', data=df, kind='kde').annotate(stats.pearsonr)

In [ ]:
sns.jointplot(x="BPXSY1", y="BPXDI1", kind='kde', data=df).annotate(stats.pearsonr)

In [ ]:
jp = sns.jointplot(x="BPXSY1", y="BPXSY2", kind='kde', data=df).annotate(stats.pearsonr)

In [ ]:
df['RIAGENDRx']=df.RIAGENDR.replace({1:'Male', 2:'Female'})
sns.FacetGrid(df, col='RIAGENDRx')

In [ ]:
sns.FacetGrid(df, row='RIAGENDRx')

In [ ]:
sns.FacetGrid(df, col='RIAGENDRx').map(plt.scatter, 'BMXLEG','BMXARML').add_legend()

Consistent with the scatterplot, a slightly weaker correlation between arm length and leg length in women (compared to men) can be seen by calculating the correlation coefficient separately within each gender.  

The '`corr`' method of a dataframe calculates the correlation coefficients for every pair of variables in the dataframe.  This method returns a "correlation matrix", which is a table containing the correlations between every pair of variables in the data set.  Note that the diagonal of a correlation matrix always contains 1's, since a variable always has correlation 1 with itself.  The correlation matrix is also symmetric around this diagonal, since the correlation between two variables '`X`' and '`Y`' does not depend on the order in which we consider the two variables.  

In the results below, we see that the correlation between leg length and arm length in men is 0.50, while in women the correlation is 0.43.

In [ ]:
print(df.loc[df.RIAGENDRx=='Female',['BMXLEG','BMXARML']].dropna().corr())

In [ ]:
print(df.loc[df.RIAGENDRx=='Male',['BMXLEG','BMXARML']].dropna().corr())

Next we look to stratifying the data by both gender and ethnicity. This results in 2 x 5 = 10 total strata, since there are 2 gender strata and 5 ethnicity strata. These scatterplots reveal differences in the means as well a diffrences in the degree of association (correlation) between different pairs of variables. We see that although some ethnic groups tend to have longer/shorter arms and legs than others, the relationship between arm length and leg length within genders is roughly similar across the ethnic groups.

One notable observation is that ethnic group 5, which consists of people who report being multi-racial or are of any race not treated as a separate group (due to small sample size), the correlation between arm length and leg length is stronger, especially for men. This is not surprising, as greater heterogeneity can allow correlations to emerge that are indiscernible in more homogeneous data.

In [ ]:
_ = sns.FacetGrid(df, col="RIDRETH1",  row="RIAGENDRx").map(plt.scatter, "BMXLEG", "BMXARML", alpha=0.5).add_legend()

### Categorical bivariate data

In this section we discuss some methods for working with bivariate data that are categorical.  We can start with a contingency table, which counts the number of people having each combination of two factors.  To illustrate, we will consider the NHANES variables for marital status and education level.

First, we create new versions of these two variables using text labels instead of numbers to represent the categories.  We also create a new data set that omits people who responded "Don't know" or who refused to answer these questions.

In [ ]:
df["DMDEDUC2x"] = df.DMDEDUC2.replace({1: "<9", 2: "9-11", 3: "HS/GED", 4: "Some college/AA", 5: "College", 
                                       7: "Refused", 9: "Don't know"})
df["DMDMARTLx"] = df.DMDMARTL.replace({1: "Married", 2: "Widowed", 3: "Divorced", 4: "Separated", 5: "Never married",
                                      6: "Living w/partner", 77: "Refused"})
db = df.loc[(df.DMDEDUC2x != "Don't know") & (df.DMDMARTLx != "Refused"), :]

Now we can create a contingency table, counting the number of people in each cell defined by a combination of education and marital status.

In [ ]:
db.head()

In [ ]:
db.shape

In [ ]:
df.shape

In [ ]:
df["DMDEDUC2x"].value_counts()

In [ ]:
db["DMDEDUC2x"].value_counts()

In [ ]:
df.DMDEDUC2x.value_counts().sum()

In [ ]:
db.DMDEDUC2x.value_counts().sum()

In [ ]:
df.DMDMARTLx.value_counts()

In [ ]:
df.DMDMARTLx.value_counts().sum()

In [ ]:
db.DMDMARTLx.value_counts().sum()

In [ ]:
x=pd.crosstab(db.DMDEDUC2x, db.DMDMARTLx)
x

In [ ]:
y=pd.crosstab(df.DMDEDUC2x, df.DMDMARTLx)
y

The results will be easier to interpret if we normalize the data. A contingency table can be normalized in three ways -- we can make the rows sum to 1, the columns sum to 1, or the whole table sum to 1. Below we normalize within rows. This gives us the proportion of people in each educational attainment category who fall into each group of the marital status variable.

The modal (most common) marital status for people within each educational attainment group is "married". However quantitatively, the proportion of people who are married varies substantially, and is notably higher for college graduates (around 61%) compared to groups with lower educational attainment.

In [ ]:
x.apply(lambda z: z/z.sum(), axis=1)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(x.apply(lambda z: z/z.sum(), axis=1), annot=True, linewidths=0.5)

In [ ]:
sns.heatmap?

We can also normalize within the columns instead of normalizing within the rows. This gives us the proportion of people with each marital status group who have each level of educational attainment.

In [ ]:
x.apply(lambda z: z/z.sum(), axis=0)

We see here that the plurality of divorced people have some college but have not graduated from college, while the plurality of married people are college graduates.

It is quite plausible that there are gender differences in the relationship between educational attainment and marital status. Therefore we can look at the proportion of people in each marital status category, for each combination of the gender and education variables. This analyses yields some interesting trends, notably that women are much more likely to be widowed or divorced than men (e.g. women in the HS/GED group are around 3 times more likely to be widowed than men in the HS/GED group).

In [ ]:
# The following line does these steps, reading the code from left to right:
# 1 Group the data by every combination of gender, education, and marital status
# 2 Count the number of people in each cell using the 'size' method
# 3 Pivot the marital status results into the columns (using unstack)
# 4 Fill any empty cells with 0
# 5 Normalize the data by row
dc=db.groupby(["RIAGENDRx", "DMDEDUC2x", "DMDMARTLx"]).size().unstack().fillna(0).apply(lambda x: x/x.sum(), axis=1)
dc

In [ ]:
plt.figure(figsize=(18,18))
sns.heatmap(dc, annot=True, linewidths=0.5)

One factor behind the greater number of women who are divorced and widowed could be that women live longer than men. To minimize the impact of this factor, we can recalculate the above table using a few narrow bands of ages. To simplify here, we collapse the marital status data to characterize people as being either "married" or "unmarried" This allows us to focus on the marriage rate, which is a widely-studied variable in social science research.

There are a number of intriguing results here. For example, the marriage rate seems to drop as college-educated people get older (e.g. 71% of college educated women between 49 and 50 are married, but only 65% of college educated women between 50 and 59 are married, an even larger drop occurs for men). However in people with a HS/GED level of education, the marriage rate is higher for older people (although it is lower compared to the college educated sample). There are a number of possible explanations for this, for example, that remarriage after divorce is less common among college graduates.

In [ ]:
dx = db.loc[(db.RIDAGEYR >= 40) & (db.RIDAGEYR < 50)]
a = dx.groupby(["RIAGENDRx", "DMDEDUC2x", "DMDMARTLx"]).size().unstack().fillna(0).apply(lambda x: x/x.sum(), axis=1)

In [ ]:
a

In [ ]:
print(a.loc[:, ["Married"]])

In [ ]:
a.loc[:, ["Married"]].unstack()

In [ ]:
plt.figure(figsize=(12,5))
sns.heatmap(a.loc[:, ["Married"]].unstack(), annot=True, linewidths=0.5)

In [ ]:
dx = db.loc[(db.RIDAGEYR >= 50) & (db.RIDAGEYR < 60)]
b = dx.groupby(["RIAGENDRx", "DMDEDUC2x", "DMDMARTLx"]).size().unstack().fillna(0).apply(lambda x: x/x.sum(), axis=1)


In [ ]:
print(b.loc[:, ["Married"]].unstack())

We conclude this section by noting that marital status is associated with many factors, including gender and eduational status, but also varies strongly by age and birth cohort.  For example, it is unlikely for young people to be widowed, and it is less likely for older people to be "never married", since a person can transition from "never married" into one of the other categories, but can never move back.  Below we will consider the role of age in more detail, and later in the course we will revisit these questions using more sophisticated analytic methods that can account for all of these factors simultaneously.  However, since NHANES is a cross-sectional study, there are certain important questions that it cannot be used to answer.  For example, while we know each person's current marital status, we do not know their full marital history (e.g. how many times and at what ages they were married or divorced).

### Mixed categorical and quantitative data

Another situation that commonly arises in data analysis is when we wish to analyze bivariate data consisting of one quantitative and one categorical variable. To illustrate methods that can be used in this setting, we consider the relationship between marital status and age in the NHANES data.  Specifically, we consider the distribution of ages for people who are currently in each marital status category.  A natural tool in this setting is side-by-side boxplots.  Here we see some unsurprising things -- widowed people tend to be older, and never-married people tend to be younger.

In [ ]:
plt.figure(figsize=(12, 4))
a = sns.boxplot(db.DMDMARTLx, db.RIDAGEYR)

When we have enough data, a "violinplot" gives a bit more insight into the shapes of the distributions compared to a traditional boxplot.  The violinplot below is based on the same data as the boxplot above.  We can see quite clearly that the distributions with low mean (living with partner, never married) are strongly right-skewed, while the distribution with high mean (widowed) is strongly left-skewed.  The other distributions have intermediate mean values, and are approximately symmetrically distributed.  Note also that the never-married distribution has a long shoulder, suggesting that this distributions includes many people who are never-married because they are young, and have not yet reached the ages when people typically marry, but also a substantial number of people will marry for the first time anywhere from their late 30's to their mid-60's.

In [ ]:
plt.figure(figsize=(12, 4))
a = sns.violinplot(df.DMDMARTLx, df.RIDAGEYR)